In [ ]:
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime
import calendar
import time
import matplotlib.pyplot as plt
import math

#from sklearn import linear_model
#from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier

# scaling the input feature
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

from joblib import load
from joblib import dump

In [ ]:
train_fname = './data/processed/crime_data/fullTrainTestCrimeData_train_weather_by_hourgroup_police_rand_2019_06_27.csv'
valid_fname = './data/processed/crime_data/fullTrainTestCrimeData_valid_weather_by_hourgroup_police_rand_2019_06_27.csv'
tests_fname = './data/processed/crime_data/fullTrainTestCrimeData_tests_weather_by_hourgroup_police_rand_2019_06_27.csv'

In [ ]:
df_train = pd.read_csv(train_fname)
df_train = df_train.drop(columns=["Unnamed: 0"])

#df_train.boxplot(column="Value")
# remove outliers
ind_to_drop_outlier = df_train[df_train["Value"]>60].index
df_train = df_train.drop(ind_to_drop_outlier, axis=0)

df_valid = pd.read_csv(valid_fname)
df_valid = df_valid.drop(columns=["Unnamed: 0"])

df_test = pd.read_csv(tests_fname)
df_test = df_test.drop(columns=["Unnamed: 0"])

#df_uniq_id["DATE"] = df_uniq_id["OCCURRED_ON_DATE"].apply(lambda x: x.split(" ")[0])
df_train["Value_Occ"] = df_train["Value"].apply(lambda x: 1 if x>0 else x)
df_valid["Value_Occ"] = df_valid["Value"].apply(lambda x: 1 if x>0 else x)
df_test["Value_Occ"]  = df_test["Value"].apply(lambda x: 1 if x>0 else x)